<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Imports
import gzip
import os
import pandas as pd
import numpy as np
import torch
import basic_model
import random
from collections import Counter
from tqdm import tqdm
import ujson as json
from torch.autograd import Variable
import util
import args
import pprint

args = args.get_setup_args()
vars(args)

{'vocab_path': '/home/mouadh/Desktop/insuranceQA/V2/vocabulary',
 'label_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.label2answer.token.encoded',
 'train_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.question.anslabel.token.500.pool.solr.train.encoded',
 'test_path': '/home/mouadh/Desktop/insuranceQA/V2/InsuranceQA.question.anslabel.token.500.pool.solr.test.encoded',
 'glove_path': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/glove.840B.300d.txt',
 'glove_dim': 300,
 'glove_num_vecs': 2196017,
 'hidden_size': 100,
 'word_emb_file': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/word_embedding',
 'word2idx_file': '/home/mouadh/Desktop/insuranceQA/glove.840B.300d/word2idx',
 'seed': 0,
 'create_matrix_embedding': True,
 'lr': 0.5,
 'num_epochs': 30,
 'drop_prob': 0.2,
 'margin': 0.2,
 'batch_size': 128,
 'use_glove': False,
 'embd_size': 200}

In [2]:
# Fix the random seed to have reprodusable results
random.seed(args.seed)
np.random.seed(args.seed)
#torch.manual_seed(args.seed)
#torch.cuda.manual_seed_all(args.seed)

In [3]:
# Load answers labels and answer label with text
id2w, l2a, l2at = util.load_vocabulary(args.vocab_path, args.label_path)

# Create word to index vocabulary
w2i = {w: i for i, w in enumerate(id2w.values(), 1)}
# Add pad to the vocabulary
PAD = '<PAD>'
w2i[PAD] = 0

vocab_size = len(w2i)
args.vocab_size = vocab_size
print('vocab_size:', vocab_size)

vocab_size: 68581


In [4]:
train = util.load_data_train(args.train_path, id2w, l2at)

In [25]:
test = util.load_data_test(args.test_path, id2w, l2at)

In [5]:
print('n_train:', len(train))
print('n_test:', len(test))

n_train: 21325
n_test: 1988


In [6]:
if args.use_glove:
    if not args.create_matrix_embedding:
        # Create word embedding and word2idx for glove
        print("Creating word embedding")
        word_emb_mat, word2idx_dict = util.get_embedding('word', emb_file=args.glove_path,
                                        vec_size=args.glove_dim, num_vectors=args.glove_num_vecs)
        util.save(args.word_emb_file, word_emb_mat, message="word embedding")
        util.save(args.word2idx_file, word2idx_dict, message="word dictionary")
    else:
        # Get embeddings
        print('Loading word vectors embeddings...')
        word_vectors = util.torch_from_json(args.word_emb_file)
        print("Loading word to index dictionary")
        word2idx_dict = json.load(open(args.word2idx_file))
else:
    word2idx_dict = w2i

In [7]:
train[0][0]

['Is', 'Disability', 'Insurance', 'Required', 'By', 'Law', '?']

In [8]:
word2idx_dict['?']

50370

In [9]:
util.make_vector([train[0][0]], w2i, 20)

tensor([[66164, 54421, 29876, 56902, 59631, 57715, 50370,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])

In [11]:
def train_model(model, data, optimizer, word2idx_dict, args, batch_size = 128):
    for epoch in range(1): #args.num_epoch:
        model.train()
        random.shuffle(data)
        losses = []
        for i, d in enumerate(tqdm(data)):
            q, pos, negs = d[0], d[1], d[2]
            #pos = process_words(pos, word2idx_dict)
            #q = process_words(q, word2idx_dict)
            
            vec_q = util.make_vector([q], word2idx_dict, len(q))
            vec_pos = util.make_vector([pos], word2idx_dict, len(pos))
            pos_sim = model(vec_q, vec_pos)

            for _ in range(100):
                neg = random.choice(negs)
                #neg = process_words(neg, word2idx_dict)
                vec_neg = util.make_vector([neg], word2idx_dict, len(neg))
                neg_sim = model(vec_q, vec_neg)
                #print('positive similarity {}, negative similarity{}'.format(pos_sim, neg_sim))
                loss = util.loss_fn(pos_sim, neg_sim, args.margin)
                if loss.data[0] != 0:
                    losses.append(loss)
                    break

            if len(losses) == batch_size or i == len(data) - 1:
                loss = torch.mean(torch.stack(losses, 0).squeeze(), 0)
                #print(loss)
                print("epoch: {}, iteration: {}, loss: {}".format(epoch, i, loss.item()))
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                losses = []
    return model

In [12]:
def lowercase(tokens):
    return [token.lower() for token in tokens]

In [13]:
from nltk.tokenize import RegexpTokenizer
tokenizer_punct = RegexpTokenizer(r'\w+')
import itertools

def remove_special_chars(tokens):
    return list(itertools.chain(*[tokenizer_punct.tokenize(token) for token in tokens]))


In [14]:
def remove_words_not_glove(tokens, word2idx_dict):
    filteredlist = []
    for token in tokens:
        try: 
            word2idx_dict[token]
            filteredlist.append(token)
        except:
            pass
    return filteredlist

In [15]:
remove_words_not_glove(['websitelink', 'hello'], word2idx_dict)

[]

In [16]:
def process_words(tokens, word2idx_dict):
    tokens = lowercase(tokens)
    tokens = remove_special_chars(tokens)
    tokens = remove_words_not_glove(tokens, word2idx_dict)
    return tokens

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

if args.use_glove:
    model = basic_model.QA_LSTM(args , word_vectors)
else: 
    model = basic_model.QA_LSTM(args)

if torch.cuda.is_available():
    model.cuda()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))

In [18]:
model = train_model(model, train, optimizer, word2idx_dict, args, args.batch_size)

  1%|          | 125/21325 [00:06<19:39, 17.98it/s]

epoch: 0, iteration: 127, loss: 0.20360736548900604


  1%|          | 255/21325 [00:45<19:30, 18.00it/s]   

epoch: 0, iteration: 255, loss: 0.19979801774024963


  2%|▏         | 383/21325 [01:24<18:14, 19.14it/s]   

...

100%|█████████▉| 21280/21325 [2:08:44<00:07,  6.36it/s]

epoch: 0, iteration: 21280, loss: 0.1609634906053543


100%|█████████▉| 21323/21325 [2:09:25<00:00, 10.08it/s]

epoch: 0, iteration: 21324, loss: 0.15466032922267914


100%|██████████| 21325/21325 [2:09:35<00:00,  1.47s/it]


In [32]:
def test_model(model, data):
    acc, total = 0, 0
    for d in data:
        q = d[0]
        print('Questions:', ' '.join(q))
        labels = d[1]
        cands = d[2]

        # preprare answer labels
        label_indices = [cands.index(l) for l in labels if l in cands]

        # build data
        q = util.make_vector([q], w2i, len(q))
        cands = [l2at[c] for c in cands] # id to text
        max_cand_len = min(200, max([len(c) for c in cands]))
        cands = util.make_vector(cands, w2i, max_cand_len)

        # predict
        scores = [model(q, c.unsqueeze(0)).data[0] for c in cands]
        pred_idx = np.argmax(scores)
        if pred_idx in label_indices:
            print('Correct Prediciton')
            acc += 1
        else:
            print('Wrong Prediction')
        total += 1
    print('Test Acc:', 100*acc/total, '%')

In [33]:
test_model(model, test)

q How To Get Health Insurance When Pregnant ?
wrong
q Can A Corporation Pay For Disability Insurance ?
wrong
q Will Homeowners Insurance Cover A Tree Falling On My Car ?
wrong
q Where To Start Looking For Health Insurance ?
wrong
q What Is Medigap Blue ?
wrong
q Is Workers Comp The Same As Disability Insurance ?
wrong
q Is Disability Insurance Necessary ?
wrong
q How Much Optional Life Insurance Do I Need ?
wrong
q What Is A Health Insurance Claim Form 1500 ?
wrong
q Can You Borrow Money From Renters Insurance ?
wrong
q Where To Get Home Insurance Quote ?
wrong
q Why Buy Variable Life Insurance ?
wrong
q How Long To Plan For Retirement ?
wrong
q Who Is The Least Expensive Auto Insurance ?
wrong
q What Is Centers For Medicare And Medicaid ?
wrong
q How To File A Renters Insurance Claim ?
wrong
q What Do I Need To Know About Buying Life Insurance ?
wrong
q What Is The Biggest Car Insurance Company In Us ?
wrong
q Does Home Owners Insurance Cover Pool Damage ?
wrong
q Why Do Smokers Pay M

wrong
q Can I Get Life Insurance If I Have Epilepsy ?
wrong
q What Does Health Insurance Protect You From ?
wrong
q Can Felons Buy Life Insurance ?
wrong
q How Many People Go Without Health Insurance ?
wrong
q What 's The Difference Between Whole Life Insurance And Endowment Insurance ?
wrong
q Does Renters Insurance Cover Hurricanes
wrong
q Can Health Insurance Charge More For Smokers ?
wrong
q Why Is Dental Care Not Covered By Medicare ?
wrong
q Where Can I Get Health Insurance In Georgia ?
wrong
q What Does Medicare Voucher System Mean ?
wrong
q Does Life Insurance Expire At A Certain Age ?
wrong
q Does Life Insurance Cover AD&D ?
wrong
q What Glucose Meter Is Covered By Medicare ?
wrong
q Who Is Western National Life Insurance Company ?
wrong
q How Much Is Medicare Per Year ?
wrong
q How Much Is A Term Life Insurance Policy ?
wrong
q Why Was Universal Life Insurance Created ?
wrong
q How Is Medicare Advantage Affected By Obamacare ?
wrong
q What The Best Life Insurance Policy To Ge

wrong
q Is Homeowners Insurance A Waste Of Money ?
wrong
q What Is The Best Medigap Insurance Company ?
wrong
q How To File A Claim With Home Insurance ?
wrong
q How To Write An Appeal Letter To Health Insurance ?
wrong
q Can You Negotiate Life Insurance Rates ?
wrong
q How Soon Should I Apply For Medicare ?
wrong
q Does My Renters Insurance Cover My Roommate ?
wrong
q Does Renters Insurance Cover A Broken Refrigerator ?
wrong
q Who Should Enroll In Medicare Part B ?
wrong
q How Long Can Children stay On Parents Auto Insurance ?
wrong
q Does AAA Offer Homeowners Insurance ?
wrong
q Who Has The Best Whole Life Insurance Rates ?
wrong
q What Does Dave Ramsey Say About Cancer Insurance ?
wrong
q Will Annuity Rates Improve In The Future ?
wrong
q When Do Companies Have To Offer Health Insurance ?
wrong
q Does Homeowners Insurance Cover Raccoon Damage ?
wrong
q What Does Annuity Due Mean ?
wrong
q What Does The Bible Say About Life Insurance ?
wrong
q Do You Need Home Insurance Before Closi

KeyboardInterrupt: 

In [40]:
# Save model
torch.save(model.state_dict(), 'first_model')


In [41]:
# Test load
m = basic_model.QA_LSTM(args)
m.load_state_dict(torch.load('first_model'))


IncompatibleKeys(missing_keys=[], unexpected_keys=[])